# Réalisation d'une analyse statistique avec le langage R
Dans un premier temps, nous allons récupérer les données dans le CSV

In [ ]:
# Charger les données
source("charger.R")
mondata <- charger(2105482)
Y <- mondata$Sales
X1 <- mondata$Price
X2 <- mondata$Advertising
X3 <- mondata$Region

In [ ]:
# Afficher les données
mondata

# Histogramme des ventes

In [ ]:
hist(Y, col="gold",border="red", main=paste("Histogramme des ventes"),
     xlab="Ventes",ylab="Fréquences")

# Diagramme de Tukey pour les ventes

In [ ]:
boxplot(Y,
horizontal=TRUE,
col="gold",border="red",
xlab="Nombre de vente")

# Droite de Henry

In [ ]:
qqnorm(Y, col ="gold", main=paste("Droite de Henry"))
qqline(Y, col ="red")


# Test de normalité (Shapiro-Wilk)

In [ ]:
shapiro.test(Y)

# Interprétation du test de normalité (Shapiro-Wilk)
La statistique W représente la correlation entre les quantiles de notre variable de vente et les quantiles théoriques d'une normale de moyenne 0 et de variance 1.
Dans notre cas w = 0.99463 ce qui est très prêt de 1 donc les quantiles de notre variable sont très prêt de ce d'une normale.
De plus notre p-value = 0.7128 est très élevée comparativement à 0.05
Nous pouvons donc dire que nos données sont conformes à une distribution normale  

# Tableau de statistique descriptive

In [ ]:
mesureStatistique = data.frame(mondata=c("Sales"), 
                     moyenne=NA, écartType=NA, q1=NA, mediane=NA,
                     q3=NA, IntervalConfiance=NA)
# moyenne,
mesureStatistique$moyenne = sapply(2, function(i) mean(Y)) 
# quartiles q1, q2 = mediane, q3 
mesureStatistique[1, c("q1", "mediane", "q3")] = quantile(Y, probs = c(0.25,0.5,0.75))
# écart type,
mesureStatistique$écartType = sapply(2, function(i) sd(Y)) 
# intervalle de confiance pour la moyenne
ci <- t.test(Y, conf.level = 0.95)$conf.int
# cat("95% Confidence Interval: [", ci[1], ", ", ci[2], "]\n")
mesureStatistique$IntervalConfiance = sprintf("[%f, %f]", ci[1], ci[2])

# Afficher le resultat
mesureStatistique

# PARTIE C: Meilleur modèle

## Étude du modèle 1
Y = beta0 + beta1*X1 + epsilon

### Graphe de regression

In [ ]:
n <- length(X1) # Taille de l'échantillon
X1_mean <- mean(X1) # Moyenne de X1
Y_mean <- mean(Y) # Moyenne de Y
X1_diff <- X1 - X1_mean # Différences entre les valeurs de X1 et sa moyenne
Y_diff <- Y - Y_mean # Différences entre les valeurs de Y et sa moyenne
X1_diff_square <- X1_diff^2 # Carré des différences de X1
X1_diff_Y_diff <- X1_diff * Y_diff # Produit des différences entre X1 et sa moyenne et entre Y et sa moyenne

# Estimation des coefficients de la régression linéaire
beta1 <- sum(X1_diff_Y_diff) / sum(X1_diff_square) # Coefficient beta1
beta0 <- Y_mean - beta1 * X1_mean # Coefficient beta0

# Préparation des données pour le graphique
X1_range <- range(X1) # Plage de valeurs de X1
Y_pred <- beta0 + beta1 * X1_range # Prédiction des valeurs de Y en fonction de X1

# Tracer le graphique
plot(X1, Y, pch = 16, col = "gold", xlab = "Prix", ylab = "Ventes", main = "Régression linéaire vente-prix")
lines(X1_range, Y_pred, col = "red", lwd = 2) # Tracer 

### Tableau d'analyse de la variance

In [ ]:
modele1 <- lm(Y ~ X1)
summary(modele1)
anova(modele1)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele1)

### Interval de confiance de beta0 et beta1

In [ ]:
conf_int <- confint(modele1)
cat("Intervalle de confiance pour beta0 (intercept): ", conf_int["(Intercept)","2.5 %"], " - ", conf_int["(Intercept)","97.5 %"], "\n")
cat("Intervalle de confiance pour beta1 (pente): ", conf_int["X1","2.5 %"], " - ", conf_int["X1","97.5 %"], "\n")

## Étude du modèle 2
Y = beta0 + beta1*X2 + epsilon

### Graphe de regression

In [ ]:
n <- length(X2) # Taille de l'échantillon
X2_mean <- mean(X2) # Moyenne de X2
Y_mean <- mean(Y) # Moyenne de Y
X2_diff <- X2 - X2_mean # Différences entre les valeurs de X2 et sa moyenne
Y_diff <- Y - Y_mean # Différences entre les valeurs de Y et sa moyenne
X2_diff_square <- X2_diff^2 # Carré des différences de X2
X2_diff_Y_diff <- X2_diff * Y_diff # Produit des différences entre X2 et sa moyenne et entre Y et sa moyenne

# Estimation des coefficients de la régression linéaire
beta1 <- sum(X2_diff_Y_diff) / sum(X2_diff_square) # Coefficient beta1
beta0 <- Y_mean - beta1 * X2_mean # Coefficient beta0

# Préparation des données pour le graphique
X2_range <- range(X2) # Plage de valeurs de X2
Y_pred <- beta0 + beta1 * X2_range # Prédiction des valeurs de Y en fonction de X2

# Tracer le graphique
plot(X2, Y, pch = 16, col = "gold", xlab = "Publicites", ylab = "Ventes", main = "Régression linéaire vente-publicite")
lines(X2_range, Y_pred, col = "red", lwd = 2) # Tracer 

### Tableau d'analyse de la variance

In [ ]:
modele2 <- lm(Y ~ X2)
summary(modele2)
anova(modele2)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele2)

### Interval de confiance de beta0 et beta1

In [ ]:
conf_int <- confint(modele2)
cat("Intervalle de confiance pour beta0 (intercept): ", conf_int["(Intercept)","2.5 %"], " - ", conf_int["(Intercept)","97.5 %"], "\n")
cat("Intervalle de confiance pour beta1 (pente): ", conf_int["X2","2.5 %"], " - ", conf_int["X2","97.5 %"], "\n")

## Étude du modèle 3
Y = beta0*(X1^beta1)*exp(epsilon)

### Graphe de regression

In [ ]:
model_function <- function(X1, beta0, beta1) {
  return(beta0 * (X1^beta1))
}

# Estimer les paramètres du modèle
model1 <- nls(Y ~ model_function(X1, beta0, beta1),
           start = list(beta0 = 1, beta1 = 1))

# Obtenir les valeurs prédites du modèle
predicted <- predict(model1)

# Tracer le graphe
plot(X1, Y, main = "Régression non linéaire",
     xlab = "X1", ylab = "Y", pch = 16, col = "gold")
lines(X1, predicted, col = "red", lwd = 2)
legend("topleft", legend = "Y = beta0*(X1^beta1)",
       col = "red", lwd = 2, bty = "n")

### Tableau d'analyse de la variance

In [ ]:
summary(modele1)
anova(modele1)

### Analyse des résidus

In [ ]:
par(mfrow = c(2,2))
plot(modele1)

### Interval de confiance de beta0 et beta1

In [ ]:
# Obtenir les intervalles de confiance des coefficients
conf_int <- confint(model1) # Calculer les intervalles de confiance

# Afficher les résultats
cat("Intervalle de confiance pour beta0 : ", conf_int["beta0","2.5 %"], " - ", conf_int["beta0","97.5 %"], "\n")
cat("Intervalle de confiance pour beta1 : ", conf_int["beta1","2.5 %"], " - ", conf_int["beta1","97.5 %"], "\n")

# PARTIE C: Meilleur modèle

# PARTIE C: Meilleur modèle

# PARTIE C: Meilleur modèle

# PARTIE C: Meilleur modèle